In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
filename="./dataset/test.csv"

# import the test data 
from pandas import read_csv
f=open(filename,encoding='UTF-8')
test=read_csv(f)
print(test)

    buying  maint  doors persons lug_boot safety
0      low  vhigh      3    more      med    med
1    vhigh   high      2    more    small   high
2    vhigh    med  5more       4      med   high
3      med    med  5more       4    small    med
4     high    med      4       4    small   high
..     ...    ...    ...     ...      ...    ...
474    low  vhigh      4       4      big   high
475    med  vhigh      3    more      big   high
476  vhigh    low      3       4      big   high
477    med  vhigh      2       2      med    med
478    med  vhigh      2       4      med    low

[479 rows x 6 columns]


In [6]:
filename="./dataset/training.csv"
f=open(filename,encoding='UTF-8')
training=read_csv(f)
print(training)

     buying  maint  doors persons lug_boot safety evaluation
0      high    low  5more       2      med    med      unacc
1       med    low  5more       4    small    low      unacc
2       med    med      4    more      med    low      unacc
3       low    med      3    more      med    low      unacc
4       low  vhigh      3    more      big    low      unacc
...     ...    ...    ...     ...      ...    ...        ...
1110    med  vhigh  5more    more    small    low      unacc
1111  vhigh   high      4       4      big    med      unacc
1112  vhigh   high      3    more      med    low      unacc
1113    med    med      4       2      big    med      unacc
1114    low    low  5more       4    small    low      unacc

[1115 rows x 7 columns]


In [7]:
train_target = training['evaluation'].values
train_target[train_target=='unacc']=1
train_target[train_target=='acc']=0

train_attr = training.drop('evaluation', axis=1)
train_input = pd.get_dummies(train_attr).values

In [8]:
def neuro(train_input, test, train_target, apha, bias):
    lst_res = []
    weight = np.zeros(train_input.shape[1])
#     print(weight*train_input[1])
    for i in range(0, len(train_input)):
        # The prediction result is determined by the activation function
        O = activation((weight*train_input[i]).sum()-bias)
        # Whether the weight is updated is determined by whether the predicted value is equal to the actual value
        T = train_target[i]
        # Each iteration it should update the weight by the function
        # Whether the weight is updated is determined by whether the predicted value is equal to the actual value
        weight = weight + apha*(T-O)*train_input[i]
    for i in range(0, len(test)):
        neuro_res = activation((weight*test[i]).sum()-bias)
        lst_res.append(neuro_res)
#     print(lst_res)
    # the retrun data type is a series so that geting the score more easily
    return pd.Series(lst_res)


# activation fuction
def activation(x):
    if x >=0:
        y = 1
    else:
        y = 0
#     print(y)
    return y

# test should not call this function
def scoreRes(ser_res, train_target):
    ser_target = pd.Series(train_target)
    train_res = pd.DataFrame([ser_res,ser_target]).T
    train_res.columns = ['pred', 'actl']
    train_res_des = train_res.value_counts()
    # Get the four elements in the statistical table 
    TP = train_res_des[1][1]
    FN = train_res_des[0][1]
    FP = train_res_des[1][0]
    TN = train_res_des[0][0]
    
    # case1: unacc is positive and acc is negative
    # the calculation way of the precision, recall and f_score
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f_score = 2*precision*recall/(precision+recall)
    accuracy = train_res[train_res['pred']==train_res['actl']].shape[0]/train_res.shape[0]
    print("unacc is positive and acc is negative")
    print('precision: ' + str(precision) + ', recall: ' + str(recall) + ', f_score: ' + str(f_score))

    # case2: acc is positive and unacc is negative
    # the calculation way of the precision, recall and f_score
    TP = train_res_des[0][0]
    FN = train_res_des[1][0]
    FP = train_res_des[0][1]
    TN = train_res_des[1][1]
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f_score = 2*precision*recall/(precision+recall)
    accuracy = train_res[train_res['pred']==train_res['actl']].shape[0]/train_res.shape[0]
    print("acc is positive and unacc is negative")
    print('precision: ' + str(precision) + ', recall: ' + str(recall) + ', f_score: ' + str(f_score))
    
    return f_score


In [9]:
pred = neuro(train_input, train_input, train_target, 1, 1)
scoreRes(pred, train_target)

unacc is positive and acc is negative
precision: 0.957286432160804, recall: 0.900709219858156, f_score: 0.928136419001218
acc is positive and unacc is negative
precision: 0.7366771159874608, recall: 0.8736059479553904, f_score: 0.7993197278911565


0.7993197278911565

In [10]:
# predict the test result
test_attr = pd.get_dummies(test).values
pred = neuro(train_input, test_attr, train_target, 1, 1)
pred[pred==1] = 'unacc'
pred[pred==0] = 'acc'
pred

0      unacc
1      unacc
2        acc
3        acc
4        acc
       ...  
474      acc
475      acc
476      acc
477    unacc
478    unacc
Length: 479, dtype: object

# Modify the activation fuction and define the boundary

In [18]:
import numpy as np
# data pre-processing
train_target = training['evaluation'].values
train_target[train_target=='unacc']=1
train_target[train_target=='acc']=0

train_attr = training.drop('evaluation', axis=1)
train_input = pd.get_dummies(train_attr).values

def neuro(train_input, test, train_target, apha, bias, boundary):
    lst_res = []
    weight = np.zeros(train_input.shape[1])
#     print(weight*train_input[1])
    for i in range(0, len(train_input)):
        # Each iteration it should update the weight by the function
        # Whether the weight is updated is determined by whether the predicted value is equal to the actual value
        O = activation((weight*train_input[i]).sum()-bias)
        T = train_target[i]
        weight = weight + apha*(T-O)*train_input[i]
#         Use the weights that you finally train to predict the outcome
    for i in range(0, len(train_input)):
        neuro_res = activation((weight*train_input[i]).sum()-bias)
        lst_res.append(neuro_res)
    return scoreRes(lst_res, train_target, boundary)


# activation fuction
# 
def activation(x):
    y = 1/(1+np.e**(-x))
#     print(y)
    return y

def scoreRes(lst_res, train_target, boundary):
    ser_res = pd.Series(lst_res)
    ser_unacc = ser_res[ser_res>boundary]

    ser_unacc[:]= 1
    ser_acc = ser_res[ser_res<=boundary]
    ser_acc[:]= 0
    ser_acc.index
#     print(ser_acc)
    # acc is positive and unacc is negative
    # the calculation way of the precision, recall and f_score
    ser_target = pd.Series(train_target)
    target_pos = ser_target[ser_target==1]
    target_neg = ser_target[ser_target==0]
    train_TP = len(set(target_pos.index) & set(ser_unacc.index))
    train_TN = len(set(target_pos.index) & set(ser_acc.index))
    train_FP = len(set(target_neg.index) & set(ser_unacc.index))
    train_FN = len(set(target_neg.index) & set(ser_acc.index))

#     print(ser_acc)
#     print(ser_unacc)
    precision = train_TP / (train_TP+train_FP)
    recall = train_TP / (train_TP+train_FN)
    f_score = 2*(precision*recall)/(precision+recall)
    print(f_score)
    return f_score
    # 
a = np.arange(0.01, 1, 0.3)
neuro1 = []
for i in a:
    for j in np.arange(1, 5, 0.1):
        neuro1.append(neuro(train_input, test, train_target, j, 1, i))    
x = np.array(neuro1).max()
x

0.8621219887237315
0.8618387262455058
0.8615542974781266
0.8614116434827408
0.8611254517294786
0.8608380755302639
0.8602597402597404
0.8602597402597403
0.860114404576183
0.8598228243877019
0.8598228243877021
0.8593831677992682
0.859088528025144
0.859822824387702
0.859236002093145
0.859383167799268
0.8578975171685156
0.857596611964002
0.8569909622541202
0.8577472236911687
0.8589407446250654
0.8589407446250655
0.8577472236911687
0.8556092324208265
0.8586442459274829
0.85529854760624
0.8537248504622078
0.8560727661851257
0.8566862013851892
0.8602597402597402
0.8529250956807
0.8586442459274829
0.8557640750670242
0.8581971534001055
0.8560727661851258
0.8571428571428572
0.8571428571428572
0.85529854760624
0.8598228243877017
0.8560727661851257
0.8571428571428572
0.8565333333333333
0.8560727661851257
0.8556092324208268
0.8552985476062398
0.8549865229110513
0.8546731496488383
0.8546731496488384
0.8545159545700379
0.8543584190579318
0.8546731496488386
0.8548300053966542
0.8540423223005968
0.8556

0.8621219887237315